In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [3]:
import pandas as pd
import numpy as np
import datetime
#from pandas_summary import DataFrameSummary
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model

In [4]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

#Los documentos se subieron al Colab mediante drag and drop. 

In [22]:
df.head()

,index,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Week,Day,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenSince,Promo2Since,State,file,week,trend,Date_y,Month_y,Day_y,file_DE,week_DE,trend_DE,Date_DE,State_DE,Month_DE,Day_DE,file_y,...,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,Year_y,Week_y,CompetitionDaysOpen,CompetitionMonthsOpen,Promo2Days,Promo2Weeks,StateHoliday_bool,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday_bool,BeforeStateHoliday_bool,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bool_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_bool_fw,Promo_fw
0,0,0,4,2015-07-31,5263,555,1,1.273237,0,2.144211,2,6,30,30,2,0,-0.511043,9,13,0,1,0,0,2008-09-15,1990-01-01,4,Rossmann_DE_HE,2015-08-02 - 2015-08-08,1.744361,2015-08-02,8,2,Rossmann_DE,2015-08-02 - 2015-08-08,1.743049,2015-08-02,None,8,2,Hessen,...,0.325047,9,6,3,0.611120,-1.510789,-1.626560,1021,1018,1015,31.0,15.0,10.0,0.148437,-0.147662,21.0,-0.315778,-1.859165,0,13,Hessen,2015,31,2510,24,9342,25,-0.177442,0,0,0.644376,1.072424,0,0,5,0,5,1,0,1
1,1,1,4,2015-07-31,6064,625,1,1.273237,0,2.144211,2,6,30,30,0,0,-0.593412,11,12,1,13,2,2,2007-11-15,2010-03-29,11,Rossmann_DE_TH,2015-08-02 - 2015-08-08,1.303439,2015-08-02,8,2,Rossmann_DE,2015-08-02 - 2015-08-08,1.743049,2015-08-02,None,8,2,Thueringen,...,0.170242,9,6,3,0.870277,-0.905024,-1.269897,1021,1019,1017,10.0,10.0,10.0,-0.970080,-0.147662,21.0,-0.315778,-0.493858,0,309,Thueringen,2015,31,2815,24,1950,25,-0.177442,0,0,0.965073,1.072424,0,0,5,0,5,1,0,1
2,2,2,4,2015-07-31,8314,821,1,1.273237,0,2.144211,2,6,30,30,0,0,1.002195,12,11,1,14,3,2,2006-12-15,2011-04-04,6,Rossmann_DE_NW,2015-08-02 - 2015-08-08,1.832546,2015-08-02,8,2,Rossmann_DE,2015-08-02 - 2015-08-08,1.743049,2015-08-02,None,8,2,NordrheinWestfalen,...,0.015437,10,7,4,0.870277,-0.980745,-1.320848,1022,1019,1017,31.0,14.0,10.0,-0.970080,-1.165096,21.0,-0.315778,-1.404063,0,354,NordrheinWestfalen,2015,31,3150,24,1579,25,-0.177442,0,0,0.644376,1.072424,0,0,5,0,5,1,0,1
3,3,3,4,2015-07-31,13995,1498,1,1.273237,0,2.144211,2,6,30,30,2,2,-0.587528,9,14,0,1,0,0,2009-09-15,1990-01-01,0,Rossmann_DE_BE,2015-08-02 - 2015-08-08,0.774331,2015-08-02,8,2,Rossmann_DE,2015-08-02 - 2015-08-08,1.743049,2015-08-02,None,8,2,Berlin,...,0.479853,9,7,4,0.092806,-0.980745,-1.015137,1019,1017,1014,10.0,10.0,10.0,0.036585,0.700200,21.0,-0.315778,0.416347,20,282,Berlin,2015,31,2145,24,9342,25,-0.177442,0,0,0.965073,1.072424,0,0,5,0,5,1,0,1
4,4,4,4,2015-07-31,4822,559,1,1.273237,0,2.144211,2,6,30,30,0,0,2.859029,4,20,0,1,0,0,2015-04-15,1990-01-01,9,Rossmann_DE_SN,2015-08-02 - 2015-08-08,1.479808,2015-08-02,8,2,Rossmann_DE,2015-08-02 - 2015-08-08,1.743049,2015-08-02,None,8,2,Sachsen,...,0.634658,8,6,5,-1.462136,-1.435069,-1.218945,1020,1018,1016,10.0,10.0,10.0,-0.970080,-0.147662,21.0,-0.315778,-0.493858,20,290,Sachsen,2015,31,107,3,9342,25,-0.177442,0,0,0.644376,1.072424,0,0,5,0,5,1,0,1


In [21]:
df.columns

Index(['index', 'Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open',
       'Promo', 'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week',
       'Day', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
       'CompetitionOpenSince', 'Promo2Since', 'State', 'file', 'week', 'trend',
       'Date_y', 'Month_y', 'Day_y', 'file_DE', 'week_DE', 'trend_DE',
       'Date_DE', 'State_DE', 'Month_DE', 'Day_DE', 'file_y',
       'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
       'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC', 'Max_Humidity',
       'Mean_Humidity', 'Min_Humidity', 'Max_Sea_Level_PressurehPa',
       'Mean_Sea_Level_PressurehPa', 'Min_Sea_Level_PressurehPa',
       'Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM',
       'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'Max_Gust_SpeedKm_h',
       'Precipitatio

In [27]:
#Modifico: Agrego "_bool" a 'BeforeStateHoliday', 'AfterStateHoliday', 'StateHoliday_bw', 'StateHoliday_fw'
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [25]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday']
# contin_vars = []

In [7]:
y_out_columns = ['Sales']

In [11]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [28]:
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [29]:
X_train.shape, X_val.shape

((814150, 38), (30188, 38))

In [30]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

## Categorical columns transformation

In [31]:
# categoricals_processing = 'no_categoricals'
# categoricals_processing = 'use_onehotencoding'
categoricals_processing = 'use_embeddings'


In [35]:
from google.colab import files
uploaded = files.upload()

Saving embeddings_model.hdf5 to embeddings_model.hdf5


In [36]:
if categoricals_processing == 'use_embeddings':
    embeddings_model = load_model('embeddings_model.hdf5')
    X_train = embeddings_model.predict(np.hsplit(X_train, X_train.shape[1]), verbose=1)
    X_val = embeddings_model.predict(np.hsplit(X_val, X_val.shape[1]), verbose=1)
    X_test = embeddings_model.predict(np.hsplit(X_test, X_test.shape[1]), verbose=1)

1284/1284 [==============================] - 10s 7ms/step


In [ ]:
if categoricals_processing == 'use_onehotencoding':
    # Use One Hot Encoding
    categorical_feature_mask = [col in cat_vars for col in X_train.columns]
    ohe = OneHotEncoder(categorical_features = categorical_feature_mask)
    ohe.fit(X_train)
    X_train = ohe.transform(X_train)
    X_val = ohe.transform(X_val)
    X_test = ohe.transform(X_test)

In [37]:
X_train.shape

(814150, 128)

In [38]:
X_val.shape

(30188, 128)

In [39]:
X_test.shape

(41088, 128)

## XGBoost

In [40]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

In [78]:
n_estimators=2000
learning_rate=0.5
model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, 
                     objective='reg:squarederror', n_jobs=12, max_depth=8, gamma=0.005)
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.005,
             importance_type='gain', learning_rate=0.5, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=None, n_estimators=2000,
             n_jobs=12, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [80]:
fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'rmse', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'verbose': 10,
           }

In [81]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

[0]	validation_0-rmse:0.164684
Will train until validation_0-rmse hasn't improved in 30 rounds.
[10]	validation_0-rmse:0.013238
[20]	validation_0-rmse:0.012263
[30]	validation_0-rmse:0.011897
[40]	validation_0-rmse:0.011803
[50]	validation_0-rmse:0.011614
[60]	validation_0-rmse:0.011505
[70]	validation_0-rmse:0.011482
[80]	validation_0-rmse:0.011421
[90]	validation_0-rmse:0.011376
[100]	validation_0-rmse:0.011376
[110]	validation_0-rmse:0.011369
[120]	validation_0-rmse:0.011369
[130]	validation_0-rmse:0.011369
Stopping. Best iteration:
[104]	validation_0-rmse:0.011364



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.005,
             importance_type='gain', learning_rate=0.5, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=None, n_estimators=2000,
             n_jobs=12, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [82]:
score = model.score(X_val, y_val)

In [83]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train)*max_log_y)
    y_pred = np.exp(model.predict(X_val)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test)*max_log_y)
else:
    y_pred_train = model.predict(X_train)*y_std + y_mean
    y_pred = model.predict(X_val)*y_std + y_mean
    y_pred_test = model.predict(X_test)*y_std + y_mean

In [84]:
# Train
train_RMSE = np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

In [85]:
# Validación
val_RMSE = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

In [86]:
print(score, train_RMSE, val_RMSE)

0.9095306218080262 0.10611057642422526 0.1276796449905112


# Baseline

In [87]:
import pandas as pd
sample_csv = pd.read_csv('sample_submission.csv')

In [88]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [89]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [90]:
df_test[['Store', 'Sales']].head(10)

,Store,Sales
0,0,4759.096031
1,2,6942.568678
2,6,8817.050891
3,7,5539.358418
4,8,6562.337612
5,9,5568.420918
6,10,8030.977041
7,11,7589.598214
8,12,5034.747182
9,13,5508.567394


In [91]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

,Store,Sales
543,702,0.0
676,878,0.0
840,1096,0.0
1399,702,0.0
1532,878,0.0


In [92]:
sample_csv['Sales'] = df_test['Sales']

In [93]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [94]:
sample_csv.head()

,Id,Sales
0,1,4759.096031
1,2,6942.568678
2,3,8817.050891
3,4,5539.358418
4,5,6562.337612


# Sumbit a la competición

In [95]:
sample_csv = pd.read_csv('sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_{log_output}-{n_estimators}-{learning_rate}.csv', index=False)

* Test 0
  * n_estimator: 4000
  * lr: 0,25
   * Metrics:
    1. Score: 0.8958
    2. train_RMSE: 0.09829
    3. val_RMSE: 0.14457

* Test 1
  * n_estimator: 4000
  * lr: 0,5
    * Metrics:
    1. Score: 0.9095
    2. train_RMSE: 0.1061
    3. val_RMSE: 0.1276

* Test 2
  * n_estimator: 2000
  * lr: 0,5
  * early_stopping: 30
    * Metrics:
    1. Score: 0.9095
    2. train_RMSE: 0.1061
    3. val_RMSE: 0.1276

